In [3]:
from glob import glob
import pandas as pd

In [18]:
senators = pd.read_csv("S117_members.csv")

code_to_party = {100: "democrat", 200: "republican"}
def create_senator(senator):
    name = senator["bioname"]
    node_attrs = {
        "lastname": name.split(",")[0].lower(),
        "firstname": name.split(",")[1].split()[0].lower(),
        "party": code_to_party.get(senator["party_code"], "other"),
        "state": senator["state_abbrev"],
        "icpsr": senator["icpsr"]
    }
    return pd.Series(node_attrs)
senator_nodes = senators.apply(create_senator, axis=1)

In [8]:
contribs = pd.read_csv("raw_contributions/COS2.csv")
party_lookup = {
    "D)": "democrat",
    "R)": "republican"
}
def extract_contrib(row):
    firstname = row["FirstLastP"].split(" ")[0].lower()
    lastname = row["FirstLastP"].split(" ")[1].lower()
    party = row["FirstLastP"].split("(")[1]
    party = party_lookup.get(party, "other")
    amount = row["Total"]
    contributor = row["Contrib"]
    return pd.Series({
        "firstname": firstname,
        "lastname": lastname,
        "party": party,
        "amount": amount,
        "contributor": contributor
        })
processed_contribs = contribs.apply(extract_contrib, axis=1)
file = "raw_contributions/COS2.csv"
race = file.split("/")[-1].split(".")[0]
processed_contribs = processed_contribs.assign(race=race)

,firstname,lastname,party,amount,contributor,race
0,cory,gardner,republican,124600,Votesane PAC,COS2
1,cory,gardner,republican,64961,Apollo Global Management,COS2
2,cory,gardner,republican,55979,GEO Group,COS2
3,cory,gardner,republican,51765,Blue Cross/Blue Shield,COS2
4,cory,gardner,republican,50788,Blackstone Group,COS2
5,cory,gardner,republican,44700,Elliott Management,COS2
6,cory,gardner,republican,42666,BGR Group,COS2
7,cory,gardner,republican,34554,NextEra Energy,COS2
8,cory,gardner,republican,34413,"Brownstein, Hyatt et al",COS2
9,cory,gardner,republican,34275,"Paul, Weiss et al",COS2


In [10]:
all_processed_contribs = pd.DataFrame()
for file in glob("raw_contributions/*"):
    contribs = pd.read_csv(file)
    race = file.split("/")[-1].split(".")[0]
    processed_contribs = contribs.apply(extract_contrib, axis=1)
    processed_contribs = processed_contribs.assign(race=race)
    all_processed_contribs = pd.concat([all_processed_contribs, processed_contribs])
all_processed_contribs

,firstname,lastname,party,amount,contributor,race
0,cory,gardner,republican,124600,Votesane PAC,COS2
1,cory,gardner,republican,64961,Apollo Global Management,COS2
2,cory,gardner,republican,55979,GEO Group,COS2
3,cory,gardner,republican,51765,Blue Cross/Blue Shield,COS2
4,cory,gardner,republican,50788,Blackstone Group,COS2
...,...,...,...,...,...,...
38,don,bolduc,republican,2900,Nh Leadership Pac,NHS1
39,don,bolduc,republican,2900,Nixon Peabody LLP,NHS1
40,jeremy,kauffman,other,2200,Brennan Appraisal,NHS1
41,jeremy,kauffman,other,1500,Gfy LLC,NHS1


In [ ]:
all_processed_contribs.to_csv("contributions.csv")